In [ ]:
import random
from dataclasses import dataclass
from typing import List, Dict
from collections import defaultdict
import statistics

@dataclass
class MorningActivity:
    name: str
    base_duration: float  # in minutes
    variance: float  # standard deviation in minutes
    skip_probability: float  # chance of skipping activity
    delay_if_skipped: float  # penalty in minutes if skipped

class Person:
    def __init__(self, name: str, target_departure_time: float):
        self.name = name
        self.target_departure_time = target_departure_time
        self.actual_departure_time = 0
        self.activities_completed = []
        self.activities_skipped = []
        self.survived = False
        self.cause_of_death = None
        
    def reset(self):
        self.actual_departure_time = 0
        self.activities_completed = []
        self.activities_skipped = []
        self.survived = False
        self.cause_of_death = None

class LifeSimulation:
    def __init__(self):
        # Morning routine activities with realistic parameters
        self.activities = [
            MorningActivity("Wake up to alarm", 1, 5, 0.1, 10),  # Snooze button
            MorningActivity("Bathroom", 15, 3, 0.01, 20),  # Almost never skipped
            MorningActivity("Shower", 15, 5, 0.05, 10),
            MorningActivity("Get dressed", 10, 3, 0.001, 15),
            MorningActivity("Take medicine", 2, 1, 0.15, 0),  # Can be forgotten
            MorningActivity("Make coffee", 10, 3, 0.2, 5),
            MorningActivity("Eat breakfast", 15, 5, 0.3, 0),
            MorningActivity("Feed pets", 5, 2, 0.05, 15),
            MorningActivity("Pack bag", 5, 2, 0.1, 10),
            MorningActivity("Find keys", 2, 5, 0.3, 15)  # High variance!
        ]
        
        # Traffic and accident parameters
        self.base_accident_probability = 0.01  # 1% chance in baseline scenario
        self.rushed_accident_multiplier = 2.0  # Double risk when rushed
        self.tired_accident_multiplier = 1.5  # 50% higher risk when tired
        
    def simulate_morning(self, person: Person) -> None:
        person.reset()
        total_time = 0
        tiredness = 0
        
        # Simulate each activity
        for activity in self.activities:
            if random.random() < activity.skip_probability:
                person.activities_skipped.append(activity.name)
                total_time += activity.delay_if_skipped
                if activity.name in ["Wake up to alarm", "Take medicine"]:
                    tiredness += 1
            else:
                duration = random.gauss(activity.base_duration, activity.variance)
                total_time += max(0, duration)  # Ensure no negative durations
                person.activities_completed.append(activity.name)
        
        person.actual_departure_time = total_time
        departure_delta = total_time - person.target_departure_time
        
        # Calculate accident probability based on conditions
        accident_prob = self.base_accident_probability
        
        # Modify risk based on rushing/tiredness
        if departure_delta > 10:  # More than 10 minutes late
            accident_prob *= self.rushed_accident_multiplier
        if tiredness > 1:  # Missed multiple important activities
            accident_prob *= self.tired_accident_multiplier
            
        # Determine survival
        if random.random() < accident_prob:
            person.survived = False
            if departure_delta > 10:
                person.cause_of_death = "Traffic accident (rushed)"
            elif tiredness > 1:
                person.cause_of_death = "Traffic accident (impaired)"
            else:
                person.cause_of_death = "Traffic accident (random)"
        else:
            person.survived = True

def run_simulation(num_trials: int = 10000) -> Dict:
    sim = LifeSimulation()
    person = Person("Test Subject", target_departure_time=75)  # 75 minutes target routine
    
    results = {
        'survivals': 0,
        'deaths': 0,
        'causes_of_death': defaultdict(int),
        'departure_times': [],
        'skipped_activities': defaultdict(int),
        'completed_activities': defaultdict(int)
    }
    
    for _ in range(num_trials):
        sim.simulate_morning(person)
        
        # Record results
        if person.survived:
            results['survivals'] += 1
        else:
            results['deaths'] += 1
            results['causes_of_death'][person.cause_of_death] += 1
            
        results['departure_times'].append(person.actual_departure_time)
        
        for activity in person.activities_completed:
            results['completed_activities'][activity] += 1
        for activity in person.activities_skipped:
            results['skipped_activities'][activity] += 1
    
    return results

def analyze_results(results: Dict) -> None:
    print("\n=== Simulation Results ===")
    print(f"\nTotal Trials: {results['survivals'] + results['deaths']}")
    print(f"Survival Rate: {results['survivals']/(results['survivals'] + results['deaths'])*100:.2f}%")
    
    print("\nCauses of Death:")
    for cause, count in results['causes_of_death'].items():
        print(f"- {cause}: {count} ({count/(results['deaths'])*100:.2f}% of deaths)")
    
    print("\nDeparture Time Statistics (minutes):")
    print(f"Average: {statistics.mean(results['departure_times']):.2f}")
    print(f"Median: {statistics.median(results['departure_times']):.2f}")
    print(f"Std Dev: {statistics.stdev(results['departure_times']):.2f}")
    
    print("\nMost Commonly Skipped Activities:")
    skipped_sorted = sorted(results['skipped_activities'].items(), 
                          key=lambda x: x[1], reverse=True)
    for activity, count in skipped_sorted[:5]:
        print(f"- {activity}: {count} times")

if __name__ == "__main__":
    # Run simulation
    results = run_simulation(10000)
    analyze_results(results)

In [ ]:
import random
import statistics

def simulate_morning_routine():
    """
    Simulates one morning routine and returns:
      - departure_time (minutes after a reference start time)
    """
    # Typical durations for tasks (in minutes)
    # You can adjust these to match a more realistic scenario.
    tasks = {
        "shower":        10,
        "breakfast":     10,
        "medicine":       2,
        "feed_pets":      5,
        "find_shoes":     3
    }
    
    # Introduce randomness: Each task can be a bit faster/slower.
    # We'll assume a uniform +/- 25% variation for demonstration.
    # You might choose normal distributions or something more complex.
    total_time = 0
    for task, base_duration in tasks.items():
        variation_factor = 1.0 + random.uniform(-0.25, 0.25)
        duration = base_duration * variation_factor
        
        # Random chance that a small rework is needed, e.g., dropping a shoe
        # We'll say there's a 5% chance of rework that takes 2 extra minutes
        if random.random() < 0.05:
            duration += 2
        
        # Random chance to skip a task (e.g., skip breakfast if running behind)
        # Let's say 5% chance to skip, for demonstration
        if random.random() < 0.05:
            duration = 0

        total_time += duration

    # We can add a random event for "I can't find my car keys"
    # that might add an extra random delay with 10% probability
    if random.random() < 0.10:
        total_time += random.uniform(1, 10)
    
    return total_time

def accident_probability(departure_time, normal_departure=30):
    """
    Given a person's departure time (in minutes) and the normal departure time,
    return the probability of being hit by the car.

    For demonstration:
      - If departure_time == normal_departure, accident is 100% (1.0).
      - If departure_time < normal_departure, accident probability is smaller, e.g., 0.3
      - If departure_time > normal_departure, accident probability is smaller, e.g., 0.05
    You can tune these probabilities to reflect your scenario.
    """
    if abs(departure_time - normal_departure) < 0.1:
        # Basically leaving at exactly normal time
        return 1.0
    elif departure_time < normal_departure:
        return 0.30
    else:
        return 0.05

def run_simulation(num_days=100_000):
    """
    Runs the simulation for a specified number of days.
    Returns statistics on accidents and average times.
    """
    normal_departure_time = 30  # e.g., 7:30 AM mapped to 30 min after some 7:00 AM reference
    
    accidents = 0
    departure_times = []
    
    for _ in range(num_days):
        dep_time = simulate_morning_routine()
        p_accident = accident_probability(dep_time, normal_departure_time)
        
        # Decide if an accident happened using the computed probability
        if random.random() < p_accident:
            accidents += 1
        
        departure_times.append(dep_time)

    accident_rate = accidents / num_days
    avg_departure = statistics.mean(departure_times)
    stdev_departure = statistics.pstdev(departure_times)
    
    results = {
        "num_days_simulated": num_days,
        "accidents": accidents,
        "accident_rate": accident_rate,
        "average_departure_time": avg_departure,
        "departure_time_stdev": stdev_departure
    }
    return results

# ---- Main Execution Example ----
if __name__ == "__main__":
    simulation_results = run_simulation(num_days=100000)
    print("Simulation Results:")
    print(f"  Days simulated:       {simulation_results['num_days_simulated']}")
    print(f"  Accidents observed:   {simulation_results['accidents']}")
    print(f"  Accident rate:        {simulation_results['accident_rate']:.2%}")
    print(f"  Avg departure time:   {simulation_results['average_departure_time']:.2f} minutes")
    print(f"  Std dev of departure: {simulation_results['departure_time_stdev']:.2f} minutes")